```
Muris Saab
University of San Diego
Sep 9, 2023

ADS-505 Applied Data Science for Business
Assignment 1.1: Use Case - Charles Book Club
```
--------------------------------------------

1\. Partition the data into training (60%) and validation (40%). Use seed = 1.

In [37]:
import pandas as pd
from sklearn.model_selection import train_test_split

data = pd.read_csv("CharlesBookClub.csv")
train_data, test_data = train_test_split(data, test_size=0.4, random_state=1)

data.shape, train_data.shape, test_data.shape

((4000, 24), (2400, 24), (1600, 24))

1.1. What is the response rate for the training data customers taken as a whole? What is the response rate for each of the `4 × 5 × 3 = 60` combinations of RFM categories? Which combinations have response rates in the training data that are above the overall response in the training data? (10 points)

In [44]:
overall_response_rate = train_data['Yes_Florence'].mean()
print("Overall response rate:" , overall_response_rate)

rfm_reponse_rate = train_data.groupby(['Rcode', 'Fcode', 'Mcode'])['Yes_Florence'].mean()
print("\nResponse rate for each RFM category: \n", rfm_reponse_rate.head())

rfm_above_overall = rfm_reponse_rate[rfm_reponse_rate > overall_response_rate]
print("\n Response rate above overall rate: \n", rfm_above_overall.head())



Overall response rate: 0.08833333333333333

Response rate for each RFM category: 
 Rcode  Fcode  Mcode
1      1      1        0.000000
              2        0.000000
              3        0.000000
              4        0.200000
              5        0.173913
Name: Yes_Florence, dtype: float64

 Response rate above overall rate: 
 Rcode  Fcode  Mcode
1      1      4        0.200000
              5        0.173913
       2      2        0.666667
       3      3        0.333333
              5        0.153846
Name: Yes_Florence, dtype: float64


1.2. Suppose that we decide to send promotional mail only to the “above-average” RFM combinations identified in part 1. Compute the response rate in the validation data using these combinations. **(10 points)**

In [42]:
test_rfm_response_rate = test_data.groupby(['Rcode', 'Fcode', 'Mcode'])['Yes_Florence'].mean()
test_rfm_response_rate_above_average = test_rfm_response_rate[test_rfm_response_rate > overall_response_rate]
print("Response rate in validation data for above average RFM combinarions:\n", test_rfm_response_rate_above_average)

Response rate in validation data for above average RFM combinarions:
 Rcode  Fcode  Mcode
1      1      2        0.333333
              4        0.200000
       3      2        1.000000
              5        0.187500
2      2      5        0.193548
       3      3        0.200000
              5        0.107143
3      1      1        0.250000
       2      4        0.142857
       3      4        0.108696
              5        0.157895
Name: Yes_Florence, dtype: float64


2\. The k-NN technique can be used to create segments based on product proximity to similar products of the products offered as well as the propensity to purchase (as measured by the RFM variables). For The Art History of Florence, a possible segmentation by product proximity could be created using the following variables:
* **R:** *recency* — months since last purchase
* **F:** *frequency* — total number of past purchases
* **M:** *monetary* — total money (in dollars) spent on books
* **FirstPurch:** months since first purchase
* **RelatedPurch:** total number of past purchases of related books (i.e., sum of purchases from the art and geography categories and of titles Secrets of Italian Cooking, Historical Atlas of Italy, and Italian Art)

2.1 Use the k-NN approach with uniform weights to classify cases with k = 1, 2, ..., 11, using `Florence` as the outcome variable. Based on the validation set, find the best `k`. Remember to normalize all five variables. Create a lift curve for the best k model, and report the expected lift for an equal number of customers from the validation dataset. **(12 points)**<br>
*Hint*: You can check sklearn.neighbors.KneighborsClassifier to assign different weights function.

2.2. The k-NN prediction algorithm gives a numerical value, which is a weighted average of the values of the Florence variable for the k-NN with weights that are inversely proportional to distance. Using the best k that you calculated above with k-NN classification, now run a model with k-NN prediction and compute a lift curve for the validation data. Use all 5 predictors and normalized data. 
* What is the range within which a prediction will fall? 
* How does this result compare to the output you get with the k-NN classification? 

**(10 points)**<br>
*Hint:* You can check sklearn.neighbors.KneighborsClassifier to assign different weights function.

```
The resulting graph shows the comparison of lift curves, and the printed ranges show the spread of predictions for both models. Typically, the regression model provides more nuanced predictions, while the classification model tends to be more decisive. In this comparison, we see the lift curve of the classification model (green line) slightly above the regression lift curve (blue line), indicating better performance.
```

3\. The logistic regression model offers a powerful method for modeling response because it yields well-defined purchase probabilities. The model is especially attractive in consumer-choice settings because it can be derived from the random utility theory of consumer behavior. Use the training set data of 2400 records to construct two logistic regression models with Florence as the outcome variable and each of the following sets of predictors: **(10 points)**
* The full set of 16 predictors in the dataset
* A subset of predictors that you judge to be the best

3.2. If the cutoff criterion for a campaign is a 30% likelihood of a purchase, find the customers in the validation data that would be targeted and count the number of buyers in this set. **(10 points)**

3.3. Briefly explain, in two to three paragraphs, the business objective, the data mining models used, why they were used, the model results, and your recommendations to your non-technical stakeholder team. **(18 points)**